In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv('/kaggle/input/bikesharingdemand-data/data.csv')
df.head(24)

In [ ]:
df.info()

# **Feature Engineering**

> Time features engineering:

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
df['date'] = pd.to_datetime(df['date'])

df['year'] = df['datetime'].apply(lambda x: x.year)
df['month'] = df['datetime'].apply(lambda x: x.month)
df['day'] = df['datetime'].apply(lambda x: x.dayofweek)
# df['day'] = df['day'].map({0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 1}) # Sunday=1, Saturday=7
df['hour'] = df['datetime'].apply(lambda x: x.hour)

In [ ]:
for col in ['season', 'holiday', 'workingday', 'weather']:
    df[col] = df[col].astype("category")

In [ ]:
df['weather'].value_counts()

In [ ]:
df.head()

In [ ]:
daily_cols = ['date', 'year', 'month', 'day', 'season', 'holiday', 'workingday']

for col in ['temp', 'atemp', 'humidity', 'windspeed']:
    agg_col_name = f'avg_{col}'
    df[agg_col_name] = df.groupby('date')[col].transform('mean')
    daily_cols.append(agg_col_name)
    
for col in ['casual', 'registered', 'count']:
    agg_col_name = f'tot_{col}'
    df[agg_col_name] = df.groupby('date')[col].transform('sum')
    daily_cols.append(agg_col_name)

In [ ]:
daily_df = df.loc[:, daily_cols].drop_duplicates()

In [ ]:
daily_df = daily_df.set_index('date')

In [ ]:
daily_df.head()

# **Data Exploration**

In [ ]:
annualy_demands = df.groupby('year')[['casual', 'registered', 'count']].sum()
annualy_demands

In [ ]:
plt.figure(figsize=(12, 12))
annualy_demands[['registered', 'casual']].plot(kind='bar', stacked=True)
plt.title('Annualy Demand (Mil)')
plt.show()

* 80% of the clients in 2011 was registred
* 82% of the clients in 2012 was registred
* 67% YOY growth


In [ ]:
monthly_demands = df.groupby(['year', 'month'])[['casual', 'registered', 'count']].sum()
monthly_demands[['registered', 'casual']].plot(kind='bar', stacked=True, figsize=(25,8))
plt.title('Monthly Demand')
plt.ylabel('Demand (units)')

In [ ]:
sns.boxplot(data=df,y="count",x="day",orient="v").set(ylabel='Daily Demand',title="Box Plot for Daily Demand by Day Of Week")

In [ ]:
dayofweek_demands = df.groupby('day')[['casual', 'registered', 'count']].mean()
plt.figure(figsize=(12, 12))
dayofweek_demands[['registered', 'casual']].plot(kind='bar', stacked=True)
plt.title('Average Demand by Day Of Week')
plt.show()

In [ ]:
dayofweek_demands['casual_prc'] = dayofweek_demands['casual'] / dayofweek_demands['count']
dayofweek_demands

In [ ]:
dayofweek_demands.mean()

weekdays:

In [ ]:
dayofweek_demands.iloc[0:5].mean()

weekends:

In [ ]:
dayofweek_demands.iloc[5:].mean()

* The average daily demand has similar distribution for all days of week.
* On weekend-days, the casual clients' rate is more than doubled (from 13% to 32% on weekend-days).

In [ ]:
hourly_demands = df.groupby(['year', 'hour'])[['casual', 'registered', 'count']].sum()
hourly_demands[['registered', 'casual']].plot(kind='bar', stacked=True, figsize=(25,8))
plt.title('Hourly Demand')
plt.ylabel('Demand (units)')

In [ ]:
def plot_moving_average(series, window, plot_intervals=False, scale=1):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(25,8))
    plt.title(f'Moving average for {series.name}\n window size = {window}')
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')
    
    #Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label='bounds: +- std')
        plt.plot(lower_bound, 'r--')
            
    plt.plot(series[window:], label='Actual', alpha=0.5, marker='.')
    plt.legend(loc='best')
    plt.grid(True)

In [ ]:
plot_moving_average(daily_df['tot_count'], 30, True)

In [ ]:
daily_df['season_desc'] = daily_df['season'].map({1: 'spring', 2: 'summer', 3: 'fall', 4: 'winter'})

In [ ]:
sns.boxplot(data=daily_df,y="tot_count",x="season_desc",orient="v").set(ylabel='Daily Demand',title="Box Plot for Daily Demand by Season")

In [ ]:
sns.boxplot(data=df,y="tot_count",x="weather",orient="v").set(ylabel='Hourly Demand',title="Box Plot for Hourly Demand by weather")

In [ ]:
daily_df.holiday.value_counts() # The number of holiday days is insignificant (2.8%)

# Roadmap & Assumptions:
1. I choose to analyze the aggregated daily data and optimize the **daily profit** by predicting the best max capacity for each day, since the average daily demand has similar distribution for all days of week (see figure: Box Plot for Daily Demand by Day Of Week).
2. Therefore, I had to drop the weather column although it can be meaningful predictior (see figure: Box Plot for Hourly Demand by weather).
3. To answer the question: What maximum capacity to set for each day? I will try to optimize the daily profit by predicting the best **total** demand for each day.

In [ ]:
COST = 10
R_PRICE = 14
C_PRICE = 18
R_REJECTION_COST = 1
C_REJECTION_COST = 0

# Sensativity Analysis

In [ ]:
def get_profit(max_capacity, r_demand, c_demand):
    r_prc = r_demand / (r_demand + c_demand)
    operational_cost = max_capacity * COST
    penalty = 0
    
    if r_demand + c_demand > max_capacity:
        income = (max_capacity * r_prc) * R_PRICE + (max_capacity * (1 - r_prc)) * C_PRICE
        penalty = (r_demand - (max_capacity * r_prc)) * R_REJECTION_COST + (c_demand - (max_capacity * (1 - r_prc))) * C_REJECTION_COST
    else:
        income = R_PRICE * r_demand + C_PRICE * c_demand
        
    profit = income - operational_cost - penalty
    return profit
    
                                                                         

Assuming Get Wheels could predict the exact number of the next day demand:

In [ ]:
daily_df['max_profit'] = daily_df.apply(lambda x: get_profit(x['tot_count'], x['tot_registered'], x['tot_casual']) , axis=1)

In [ ]:
for prc in [.1, .15, .2]:
    daily_df[f'profit_{int(prc*100)}_over'] = daily_df.apply(lambda x: get_profit(x['tot_count']*(1+prc), x['tot_registered'], x['tot_casual']) , axis=1)
    daily_df[f'profit_{int(prc*100)}_under'] = daily_df.apply(lambda x: get_profit(x['tot_count']*(1-prc), x['tot_registered'], x['tot_casual']) , axis=1)
profits_df = pd.DataFrame(daily_df[['profit_20_over','profit_15_over', 'profit_10_over', 'max_profit', 'profit_10_under', 'profit_15_under', 'profit_20_under']].sum(), columns=['profit'])

In [ ]:
profits_df['max_profit_ratio'] = profits_df['profit'] / profits_df.loc['max_profit'].values[0] 
t = profits_df.loc[['profit_10_over', 'max_profit', 'profit_10_under']]
t['marginal_profit'] = t.apply(lambda x: (x['max_profit_ratio']-1)/10, axis=1)
t

* The additional revenue that will be generate by increasing the maximum capacity by 1% over the total demand is -2.1%
* The additional revenue that will be generate by decreasing the maximum capacity by 1% under the total demand is -1.2%

In [ ]:
t['max_profit_ratio'].plot(kind='bar', figsize=(8, 4))
plt.title('Profit simulation')
plt.ylabel('Total Profit')

# Prediction Models
![https://photos.app.goo.gl/ubEJQU4Qz6YLWHMS7](https://photos.app.goo.gl/ubEJQU4Qz6YLWHMS7)

# Future Work:
* Predict the registered and casual clients' demand separately.
* Use the row data (demands per hour) and compare its results to the daily aggregated data.